<a href="https://colab.research.google.com/github/Arup3201/Summarization-Project-using-Pointer-Gen/blob/main/Get_To_The_Point_Summarization_with_Pointer_Generator_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [66]:
import pathlib
import re
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
path_to_cnn_stories = tf.keras.utils.get_file(
    origin="https://huggingface.co/datasets/cnn_dailymail/resolve/main/data/cnn_stories.tgz",
    extract=True
)

path_to_dailymail_stories = tf.keras.utils.get_file(
    origin="https://huggingface.co/datasets/cnn_dailymail/resolve/main/data/dailymail_stories.tgz",
    extract=True
)

375893739/375893739 [==============================] - 6s 0us/step


In [3]:
path_to_cnn_stories, path_to_dailymail_stories

('/root/.keras/datasets/cnn_stories.tgz',
 '/root/.keras/datasets/dailymail_stories.tgz')

In [4]:
!ls -l /root/.keras/datasets

total 521960
drwxr-xr-x 3 root root      4096 Jul 26 07:33 cnn
-rw-r--r-- 1 root root 158577824 Jul 26 07:33 cnn_stories.tgz
drwxr-xr-x 3 root root      4096 Jul 26 07:34 dailymail
-rw-r--r-- 1 root root 375893739 Jul 26 07:34 dailymail_stories.tgz


In [5]:
cnn_stories_dir = pathlib.Path('/root/.keras/datasets/cnn/stories')
dailymail_stories_dir = pathlib.Path('/root/.keras/datasets/dailymail/stories')

In [6]:
cnn_stories_dir, dailymail_stories_dir

(PosixPath('/root/.keras/datasets/cnn/stories'),
 PosixPath('/root/.keras/datasets/dailymail/stories'))

In [7]:
def print_filenames(dir_path, num_files=5):
  '''Prints the name of the files that are present at `dir_path`.
  Maximum `num_files` number of files are shown.

  Arguements:
    dir_path: PosixPath, pointing to the directory of which the user
              wants to prints the file names.
    num_files: int, number of files user wants to print.

  returns:
    nothing
  '''

  count = 0
  for f in dir_path.glob('*.story'):
    print(f.name)
    count += 1

    if count == num_files:
      break
  else:
    print(f"Less than {num_files} is present!")

In [8]:
print_filenames(cnn_stories_dir)

438411e10e1ef79b47cc48cd95296d85798c1e38.story
e453e379e8a70af2d3dff1c75c41b0a35edbe9cc.story
2079f35aca44978a7985afe0ddacdf02bedf98f2.story
4702f28c198223157bb8f69665b039d560eebb0f.story
db3e2ea79323a98379228b17cd3b9dec17dbd2cb.story


In [9]:
print_filenames(dailymail_stories_dir)

f4ba18635997139c751311b9f2ad18f455dd7c98.story
4a3ef32cff589c85ad0d22724e2ed747c0dacf87.story
5375ed75939108c72001b043d3b4799c47f32be9.story
fe9e57c21e21fb4ec26e394f0e92824f38d18a95.story
6a544b5cdd2384be6cc657b265d7aa2de72a99e0.story


In [113]:
# Define the global variables
dm_single_close_quote = u'\u2019' # unicode
dm_double_close_quote = u'\u201d'
END_TOKENS = ['.', '!', '?', '...', "'", "`", '"',
              dm_single_close_quote, dm_double_close_quote, ")"]

# Maximum stories to process from cnn and dailymail each
MAX_STORIES = 50000

# From the total data how to split into train, val and test
TRAIN_SIZE = 0.8
VAL_SIZE = 0.1
TEST_SIZE = 0.1

# For tokenization
VOCAB_SIZE = 20000
OOV_TOKEN = "<OOV>"

# For standardization
PAD_TOKEN = '<PAD>'
START_TOKEN = '<START>'
END_TOKEN = '<END>'

In [11]:
# Taking a sample .story file from cnn stories
sample_filename = "438411e10e1ef79b47cc48cd95296d85798c1e38.story"
sample_filedir = cnn_stories_dir

sample_filepath = sample_filedir / sample_filename
with open(sample_filepath, 'r') as f:
  sample_story = f.read()

print(sample_story)

New York (CNN) -- The U.S. population is expected to top out at close to 312.8 million people just around the time crowds gather to watch the ball drop on New Year's Eve, according to new census data released Thursday.

The figure represents a 0.7% increase from last year, adding 2,250,129 people to the U.S. population since the start of 2011, and a 1.3% increase since Census Day, April 1, 2010.

The agency estimates that beginning in January, one American will be born every eight seconds and one will die every 12 seconds.

U.S.-bound immigrants are also expected to add one person every 46 seconds.

That combination of births, deaths and migration is expected to add a single person to the U.S. population every 17 seconds, the Census Bureau said.

Meanwhile, millions are set to ring in the new year.

In New York, authorities are preparing for large crowds in Manhattan's Times Square, where Lady Gaga is expected to join Mayor Michael Bloomberg to push the button that drops the Waterford 

I am creating a function `fix_missing_period` where I am taking 2 arguements, one for the `line` for which I am checking and fixing the period and other is `end_tokens` which is a list that has all the tokens that I should consider as ending of a sentence.

These are the steps -
1. Check if line contains `@highlight`, if True then just return the line.
2. Check if line is empty, then return line as it is.
3. Check is line ends with any of the `end_tokens`, if so then return line as it is.
4. Only is none of the above conditions match then append `.` to the current line.

In [12]:
def fix_missing_period(line, end_tokens=END_TOKENS):
  '''function to fix the missing periods for some story lines which do not end with
  any of the end_tokens mentioned.

  Arguements:
    line: string, line of the story to fix the missing the period of.
    end_tokens: list of strings, all the tokens that are considered as line end.

  Returns:
    new line with fixed the ending part by adding an ending token if not present.
  '''
  if "@highlight" in line:
    return line
  elif line == "":
    return line
  elif line[-1] in end_tokens:
    return line

  return line + '.'

In [13]:
fix_missing_period(sample_story.split('\n')[0])

"New York (CNN) -- The U.S. population is expected to top out at close to 312.8 million people just around the time crowds gather to watch the ball drop on New Year's Eve, according to new census data released Thursday."

I am creating a function `split_article_summary` which will split the story into article and summary parts.

The function takes only 1 arguement and that is the `story` which will be splitted into article and summary.

The steps to follow are -
1. Split the story by new line `\n`. I will get a list of lines.
2. Strip the lines by using list comprehension.
3. Use list comprehension to make lower case each line by using `.lower()`.
4. Fix each line by adding period if there is none in that line using `fix_missing_period` function.
5. Make 2 empty list for `article` and `summary`.
6. Go through each line. In each line, I need to check 4 things,
  * line contains `@highlight` or not, if True then set `next_highlight` to `True` because the next to next line is going to be a summary line.
  * line is `""` empty or not, if True then ignore.
  * `next_highlight` is True or not, if True then append the line to `summary`.
  * If non of the ebove then append to `article`.
7. After done with filling the `article` and `summary` list with lines, join those sentences to make the whole article and summary. Here, I am using `.join()` method.

In [14]:
def split_article_summary(story):
  '''Splits the story into 2 parts, one for article and other for summary of that
  article. Returns the article and summary.

  Arguements:
    story: string file that contains both article and summary combiningly.

  Returns:
    article, summary seperately from the story.

  '''
  lines = story.split('\n')
  lines = [line.strip() for line in lines]
  lines = [line.lower() for line in lines]

  # Fix the ending period
  lines = [fix_missing_period(line) for line in lines]

  # List to contain the article and summary lines
  article = []
  summary = []

  # Indicator of whether the next line is the summary or not
  next_highlight = False

  for line in lines:
    if "@highlight" in line:
      next_highlight = True
    elif line=="":
      continue
    elif next_highlight:
      summary.append(line)
    else:
      article.append(line)

  article = ' '.join(article)
  summary = ' '.join(summary)

  return article, summary

In [15]:
split_article_summary(sample_story)

('new york (cnn) -- the u.s. population is expected to top out at close to 312.8 million people just around the time crowds gather to watch the ball drop on new year\'s eve, according to new census data released thursday. the figure represents a 0.7% increase from last year, adding 2,250,129 people to the u.s. population since the start of 2011, and a 1.3% increase since census day, april 1, 2010. the agency estimates that beginning in january, one american will be born every eight seconds and one will die every 12 seconds. u.s.-bound immigrants are also expected to add one person every 46 seconds. that combination of births, deaths and migration is expected to add a single person to the u.s. population every 17 seconds, the census bureau said. meanwhile, millions are set to ring in the new year. in new york, authorities are preparing for large crowds in manhattan\'s times square, where lady gaga is expected to join mayor michael bloomberg to push the button that drops the waterford cr

I am creating a function `get_articles_summaries` which will process each of the stories present in the directory of cnn and dailymail and return the articles, summaries in the form of list.

This function will take 2 arguements. One will be the `stories_dir` which is a Posix format string from `pathlib` library and another arguement is of `max_stories` which is the maximum number of stories that we will extract from those directories.

The process is simple. We will follow this steps -
1. Create 2 empty lists of `articles` and `summaries`.
2. Loop through all the files present in the directory `stories_dir` using `.glob` generator method.
3. Make a `count` variable which will count the number of processed strories and when it hits `max_stories`, break from the loop.
4. Inside the loop, you will open the file in `r` reading format, then just use `.read()` method to read the story.
5. Everytime after reading the story, split the article and summary part from it and then append them inside the `articles` and `summaries` list.
6. Return the 2 lists.

In [16]:
def get_articles_summaries(stories_dir, max_stories):
  '''stores the stories from stories_dir folder into a list and returns the list

  Arguement:
    stories_dir: Posix string, the directory where the stories are stored
    max_stories: maximum number of stories to store

  Returns:
    list of stories.

  '''
  articles = []
  summaries = []

  count = 0
  for f in stories_dir.glob("*.story"):
    count += 1
    with open(f, 'r') as reader:
      story = reader.read()

      article, summary = split_article_summary(story)

      articles.append(article)
      summaries.append(summary)

    if count == max_stories:
      break

  return articles, summaries

```
cnn
  stories
    438411e10e1ef79b47cc48cd95296d85798c1e38.story
    e453e379e8a70af2d3dff1c75c41b0a35edbe9cc.story
    2079f35aca44978a7985afe0ddacdf02bedf98f2.story
    4702f28c198223157bb8f69665b039d560eebb0f.story
    db3e2ea79323a98379228b17cd3b9dec17dbd2cb.story
    ...
    ...
    ...

dailymail
  stories
    f4ba18635997139c751311b9f2ad18f455dd7c98.story
    4a3ef32cff589c85ad0d22724e2ed747c0dacf87.story
    5375ed75939108c72001b043d3b4799c47f32be9.story
    fe9e57c21e21fb4ec26e394f0e92824f38d18a95.story
    6a544b5cdd2384be6cc657b265d7aa2de72a99e0.story
    ...
    ...
    ...

```

Out of all available .story files, we will only take `MAX_STORIES` number of files and then open them.

In [17]:
cnn_articles, cnn_summaries = get_articles_summaries(cnn_stories_dir, MAX_STORIES)

len(cnn_articles)

50000

In [18]:
print(f"Total no of cnn stories captured are {len(cnn_articles)}\n\n")
print(f"One of the CNN articles: {cnn_articles[0]}\n\n")
print(f"The summary of this article: {cnn_summaries[0]}\n\n")

Total no of cnn stories captured are 50000


One of the CNN articles: new york (cnn) -- the u.s. population is expected to top out at close to 312.8 million people just around the time crowds gather to watch the ball drop on new year's eve, according to new census data released thursday. the figure represents a 0.7% increase from last year, adding 2,250,129 people to the u.s. population since the start of 2011, and a 1.3% increase since census day, april 1, 2010. the agency estimates that beginning in january, one american will be born every eight seconds and one will die every 12 seconds. u.s.-bound immigrants are also expected to add one person every 46 seconds. that combination of births, deaths and migration is expected to add a single person to the u.s. population every 17 seconds, the census bureau said. meanwhile, millions are set to ring in the new year. in new york, authorities are preparing for large crowds in manhattan's times square, where lady gaga is expected to join mayo

In [19]:
dailymail_articles, dailymail_summaries = get_articles_summaries(dailymail_stories_dir,
                                                                 MAX_STORIES)

In [20]:
print(f"Total no of cnn stories captured are {len(dailymail_articles)}\n\n")
print(f"One of the CNN articles: {dailymail_articles[0]}\n\n")
print(f"The summary of this article: {dailymail_summaries[0]}\n\n")

Total no of cnn stories captured are 50000


One of the CNN articles: by. damien gayle. published:. 19:51 est, 19 july 2013. |. updated:. 20:24 est, 19 july 2013. the families of some of the 22 primary school pupils who died after eating a school lunch contaminated with pesticides have buried the children in the school playground in protest at 'government negligence'. police have arrested the school cook following the poisoning which may have been part of a political feud between two branches of the same family, a local police chief said. nearly 50 people, the majority of whom were young children, became sick at the rural school in the eastern state of bihar after eating the food. almost half died. the incident followed recent provincial elections in which arsal khan khichi lost to his cousin jehanzaeb khan khichi, police chief sadiq dogar said late thursday. buried in the playground: villagers look on as three of the children who died after eating food contaminated with pesticide are 

In [21]:
[1, 2] + [3, 4]

[1, 2, 3, 4]

In [22]:
random.seed(0) # Keeps the shuffling same as before
random.sample([1, 2, 3, 4, 5], 5)

[4, 5, 1, 2, 3]

In [23]:
sample_article, sample_summary = split_article_summary(sample_story)

In [24]:
sample_article

'new york (cnn) -- the u.s. population is expected to top out at close to 312.8 million people just around the time crowds gather to watch the ball drop on new year\'s eve, according to new census data released thursday. the figure represents a 0.7% increase from last year, adding 2,250,129 people to the u.s. population since the start of 2011, and a 1.3% increase since census day, april 1, 2010. the agency estimates that beginning in january, one american will be born every eight seconds and one will die every 12 seconds. u.s.-bound immigrants are also expected to add one person every 46 seconds. that combination of births, deaths and migration is expected to add a single person to the u.s. population every 17 seconds, the census bureau said. meanwhile, millions are set to ring in the new year. in new york, authorities are preparing for large crowds in manhattan\'s times square, where lady gaga is expected to join mayor michael bloomberg to push the button that drops the waterford cry

I am creating another function -
`split_dataset(train_size, val_size, test_size)`: I am creating this function to split the original 1,00,000 examples into 80,000 training samples, 10,000 val samples and 10,000 test samples.

In [25]:
def split_dataset(dataset, train_size, val_size, test_size):
  first_split = train_size
  second_split = train_size+val_size
  third_split = train_size+val_size+test_size
  return dataset[:first_split, :], dataset[first_split:second_split, :], dataset[second_split:third_split, :]

Utilize the 4 functions created above into one function called `make_datasets`. This function will -
1. This functions will have many argumenets and among them 2 argumenets `cnn_stories` and `dailymail_stories` are lists which has list of articles and summaries at 0 and 1 index. It means `cnn_stories[0]` is articles of cnn news and `cnn_stories[1]` is summaries of cnn news. It applies to `dailymail_stories` as well.
Objective of this step is to concatenate the cnn articles with dailymail articles and cnn summaries with dailymail summaries.
```python
[1, 2] + [3, 4] = [1, 2, 3, 4]
```

3. Convert the articles and summaries list into tensors and then concatenate them along a new axis. To create new axis I can use `tf.newaxis` in the indexing. E.g.
```python
  np.concatenate([articles[:, tf.newaxis], summaries[:, tf.newaxis]], axis=-1)
```
4. Shuffle the dataset using `random.sample` method.
```python
random.seed(seed_value) # To make sure that everytime it gives the same shuffle
random.sample(list_to_shuffle, len(list_to_shuffle))
```
5. Split the dataset into 3 parts, one for training, other for validation and last one for testing. All the tensors are of shape `(num_samples, 2)`.

In [26]:
def make_datasets(cnn_stories, dailymail_stories, train_fraction, val_fraction, test_fraction, seed_value=0):
  '''Create 3 datasets each for training, validation and testing respectively.
  This function concatenates the articles, summaries of cnn and dailymail news. After that it will tokenize
  them one by one in a loop. After it is done with the tokenization, it will shuffle the articles and
  summaries using random.sample method (although we have a helper function for it). Finally we do the
  splitting of the whole dataset. Remember here the returned values become tensors.

  Arguements:
    cnn_stories: list of 2 values, one for cnn articles and other for cnn summaries.
    dailymail_stories: list of 2 values, one for dailymail articles and other for dailymail summaries.
    train_size: float, specifying how much fraction of the original dataset to take for training.
    val_size: float, specifying how much fraction of the original dataset to take for validation.
    test_size: float, specifying how much fraction of the original dataset to take for testing.

  Returns:
    returns a tuple with 3 values inside it, `training_data`, `validation_data` and `testing_data`
    with the specified amount of data in it.
    Each one of them are tensor with shape `(num_samples, 2)`. `shape[1]=2` for article and summary.
  '''
  articles = cnn_stories[0] + dailymail_stories[0]
  summaries = cnn_stories[1] + dailymail_stories[1]

  articles = np.array(articles, dtype=object)
  summaries = np.array(summaries, dtype=object)

  dataset = np.concatenate((articles[:, tf.newaxis], summaries[:, tf.newaxis]), axis=-1)

  random.seed(seed_value)
  shuffled_indices = random.sample(list(range(dataset.shape[0])), dataset.shape[0])

  dataset = dataset[shuffled_indices, :]

  train_size = int(train_fraction * dataset.shape[0])
  val_size = int(val_fraction * dataset.shape[0])
  test_size = dataset.shape[0] - (train_size + val_size)

  training_samples, validation_samples, testing_samples = split_dataset(dataset,
                                                                        train_size,
                                                                        val_size,
                                                                        test_size)

  return (training_samples, validation_samples, testing_samples)

In [27]:
train_dataset, val_dataset, test_dataset = make_datasets([cnn_articles, cnn_summaries], [dailymail_articles, dailymail_summaries], TRAIN_SIZE, VAL_SIZE, TEST_SIZE)

In [28]:
print(f"Type of the datasets: {type(train_dataset)}\n")

print(f"Training dataset shape: {train_dataset.shape}")
print(f"Validation dataset shape: {val_dataset.shape}")
print(f"Testing dataset shape: {test_dataset.shape}\n")

print(f"First example in the training dataset looks like: \n {train_dataset[0]}\n")

Type of the datasets: <class 'numpy.ndarray'>

Training dataset shape: (80000, 2)
Validation dataset shape: (10000, 2)
Testing dataset shape: (10000, 2)

First example in the training dataset looks like: 
 ["by. sean poulter. a skin cancer warning has been sounded following evidence some big brand sun creams do not offer the protection they claim to give. as temperatures are predicted to hit the mid-70s this weekend, products by malibu, piz buin and hawaiian tropic have been given a ‘don’t buy’ rating by independent reviewer which?. the consumer champion tested 15 popular sun creams that claimed to have a sun protection factor (spf) of 30, as well as helping to shield the skin from uva rays. the end for lotion? the new drinkable liquid offers up to factor 30 protection. the three. which failed were found to have a real spf level of less than 25, while. the malibu product also fell short of the required uva standard. sun exposure is linked to premature ageing and skin cancer, with evide

Before the tokenization, we need to preprocess the text data so that it can be properly tokenized. In this step we need to choose whether we want to keep punctuations or not, whether we should keep the numbers or not and so on. There are 2 functions I will create, one for simple `standardize` and other to feed the Tokenizer class when creating the `tokenizer`. `standardize` function implements the following steps -
1. Lower case the strings passed to it. It is already done but for user data it might not be the case so, we will still perform this step.

2. Replace the single and double opening and closing quotes like `‘ → \u2018`, `’ → \u2019`, `“ → \u201c` and `” → \u201d` by `'` and `"` respectively.

3. Replace the punctutations ``['.', '?', '!', ',', ':', '-', ''', '"', '_', '(', ')', '{', '}', '[', ']', '`', ';', '...']`` by `[SPACE]punctutations`.
In this process we need to make sure that the floating point numbers like `1.78` do not become `1 .78`. To do that the correct regex expression is ``(?<!\d)\s*([!"#$£%&\'\(\)*+,-./:;<=>?@\[\]\\^_`{|}~])\s*(?!\d)``.

4. Strip the texts from extra starting or ending spaces. Finally, remove extra spaces using regex expression like `\s{2,}`.

`custom_analyzer` function which will be feed to the Tokenizer as the value for `analyzer`, has some more steps to implement -
1. Standardize the text with `standardizer`.

2. Remove unwanted spaces in between words.

3. Split the text into words which are seperated by ' '.

4. Strip each of the words in the sentence. Finally, return it.

In [131]:
# Standardize the text data
def standardizer(text):
  # Lower case the text
  text = text.lower()

  # Replace the special single and double opening and closing quotes
  text = re.sub(r'[\u2019\u2018]', "'", text)
  text = re.sub(r'[\u201c\u201d]', '"', text)

  # Add space before punctuations and ignore floating point numbers.
  text = re.sub(r'(?<!\d)\s*([!"#$£%&\'\(\)*+,-./:;<=>?@\[\]\\^_`{|}~])\s*(?!\d)',
                  r' \1 ', text)

  # Remove spaces after sentence end and other unwanted spaces from text
  text = text.strip()
  text = re.sub('\s{2,}', ' ', text)

  return text

# custom analyzer for the Tokenizer class
def custom_analyzer(text):
  text = standardizer(text)

  text = re.sub('\s{2,}', ' ', text)

  words = text.split(' ')
  words = [word.strip() for word in words]

  return words

In [129]:
sample_texts = ["I have been working on, \nbut \tnever did it in this way.",
                "U.S won the world cup and bagged 1.78 million dollars.",
                "India had M.S. Dhoni won made it this far.",
                "My email address is arupjana7365@gmail.com.",
                "It can take care of dailymail single opening quote’ also.",
                "I have 10,000 Rs in my bank"]

[standardizer(text) for text in sample_texts]

['i have been working on , but never did it in this way .',
 'u . s won the world cup and bagged 1.78 million dollars .',
 'india had m . s . dhoni won made it this far .',
 'my email address is arupjana7365@gmail . com .',
 "it can take care of dailymail single opening quote ' also .",
 'i have 10,000 rs in my bank']

In [132]:
[custom_analyzer(text) for text in sample_texts]

[['i',
  'have',
  'been',
  'working',
  'on',
  ',',
  'but',
  'never',
  'did',
  'it',
  'in',
  'this',
  'way',
  '.'],
 ['u',
  '.',
  's',
  'won',
  'the',
  'world',
  'cup',
  'and',
  'bagged',
  '1.78',
  'million',
  'dollars',
  '.'],
 ['india',
  'had',
  'm',
  '.',
  's',
  '.',
  'dhoni',
  'won',
  'made',
  'it',
  'this',
  'far',
  '.'],
 ['my', 'email', 'address', 'is', 'arupjana7365@gmail', '.', 'com', '.'],
 ['it',
  'can',
  'take',
  'care',
  'of',
  'dailymail',
  'single',
  'opening',
  'quote',
  "'",
  'also',
  '.'],
 ['i', 'have', '10,000', 'rs', 'in', 'my', 'bank']]

Now, I need to find the tokens from the articles. I need to use only training articles not any other and also I will not use summaries data because that will be my target and I won't know what type of words I will encounter when summarizing the source article. So, the only words that I know will be from the articles of training dataset. Here, I am going to use the `tensorflow.keras.preprocessing.text.Tokenizer` in short `Tokenizer` to find the tokens from the articles and then finally converting the articles into sequence of integers. One thing to remember is here we are going to use `oov_token` arguement of `Tokenizer` to mention the token we want to use for out-of-vocabulary words.

When fiting the texts on `tokenizer` make sure to remove floating point and integer numbers using the regex expression - `[+-]?[0-9]*[.]?[0-9]+`. I am making sure that tokenizer does learn the numbers because it can always be taken from the original articles data and we do not to remember them in vocab.

In [133]:
def get_tokenizer(texts, num_words, oov_token, filters = '#*+/:<=>@[\\]/^{|}~\t\n'):
  # Create the tokenizer usinf Tokenizer class
  tokenizer = Tokenizer(num_words=num_words,
                        filters=filters,
                        oov_token=oov_token,
                        analyzer=custom_analyzer)

  # Remove the numbers from the dataset so that tokenizer does not add them inside vocabulary
  texts = [re.sub(r"[+-]?[0-9]*[.]?[0-9]+", "", text) for text in texts]

  # Fit the data with fit_on_texts method
  tokenizer.fit_on_texts(texts)

  return tokenizer

In [134]:
print(f"Length the articles dataset: {len(list(train_dataset[:, 0]))}")

Length the articles dataset: 80000


In [141]:
tokenizer = get_tokenizer(list(train_dataset[:, 0]), VOCAB_SIZE, OOV_TOKEN)

In [142]:
print(f"The vocabulary for the tokenizer has a length {len(tokenizer.word_index.keys())}")
print(f"'<OOV>' word had index: {tokenizer.word_index['<OOV>']}")
print(f"'teacher' word has index: {tokenizer.word_index['teacher']}")

print(f"Text to Sequence of the first article is {tokenizer.texts_to_sequences([train_dataset[0, 0]])}")

sample_sequence = tokenizer.texts_to_sequences([train_dataset[0, 0]])
print(f"Sequence to Text of the first acrticle is {tokenizer.sequences_to_texts(sample_sequence)}")

The vocabulary for the tokenizer has a length 251628
'<OOV>' word had index: 1
'teacher' word has index: 1571
Text to Sequence of the first article is [[29, 2, 3483, 9984, 2, 7, 1468, 695, 1528, 32, 46, 6098, 309, 484, 89, 298, 1669, 1356, 16656, 96, 40, 1010, 3, 1597, 35, 872, 6, 371, 2, 24, 2311, 34, 3652, 6, 380, 3, 1, 38, 791, 4, 1328, 29, 14264, 4, 1, 1, 8, 12231, 1, 28, 46, 346, 7, 5, 157, 5, 51, 1124, 5, 4699, 29, 1496, 1, 56, 98, 2, 3, 2945, 1393, 2682, 1, 805, 1356, 16656, 14, 558, 6, 28, 7, 1356, 1597, 2506, 50, 19052, 49, 9, 1, 24, 149, 24, 1406, 6, 6143, 3, 1468, 30, 1, 7297, 2, 3, 230, 15, 1, 98, 3, 68, 1, 4983, 1953, 58, 6, 2506, 1, 1597, 2, 3, 108, 2, 56, 801, 45, 132, 6, 28, 7, 342, 19052, 663, 9, 433, 75, 1, 102, 2, 3, 14264, 1790, 67, 1105, 778, 9, 3, 1703, 1, 1997, 2, 1356, 3967, 18, 1679, 6, 7876, 9555, 8, 1468, 695, 4, 22, 484, 9, 1, 704, 97, 2150, 10, 3, 414, 270, 59, 2, 56, 98, 132, 14, 1827, 7, 186, 960, 18, 79, 4765, 9, 363, 1597, 4, 2, 22, 3, 114, 1969, 1669, 

In [143]:
train_dataset[0, 0]

"by. sean poulter. a skin cancer warning has been sounded following evidence some big brand sun creams do not offer the protection they claim to give. as temperatures are predicted to hit the mid-70s this weekend, products by malibu, piz buin and hawaiian tropic have been given a ‘don’t buy’ rating by independent reviewer which?. the consumer champion tested 15 popular sun creams that claimed to have a sun protection factor (spf) of 30, as well as helping to shield the skin from uva rays. the end for lotion? the new drinkable liquid offers up to factor 30 protection. the three. which failed were found to have a real spf level of less than 25, while. the malibu product also fell short of the required uva standard. sun exposure is linked to premature ageing and skin cancer, with evidence of 100,000 cases being diagnosed in the uk each year. which? found that paying a high price is no guarantee of better protection,. with the most expensive brand outperformed by a cheap rival. piz. buin u

The oddness you might see if you are that much familiar with `Tokenizer` class is, even though I have specified that `num_words=VOCAB_SIZE` which is `20,000` still the length of the `word_index` is more that that. Does that mean we are doing something wrong?
NO, here although tokenizer computes the word_index of all other words apart from those first 20000 words, it will not use them when we convert them into sequence. Let's look at one example to understand that.

In [146]:
list(tokenizer.word_index.keys())[21000]

'resounding'

In [154]:
sample_text = "This example is to test the above fact with the word `resounding`"
sample_sequence = tokenizer.texts_to_sequences([sample_text])

print(f"Text: {sample_text}\n\n")
print(f"Tokenized text: {tokenizer.sequences_to_texts(sample_sequence)}")
print(f"Sequence: {sample_sequence}")

Text: This example is to test the above fact with the word `resounding`


Tokenized text: ['this example is to test the above fact with the word ` <OOV> `']
Sequence: [[38, 1127, 18, 6, 834, 3, 764, 539, 22, 3, 1302, 16926, 1, 16926]]


Although `resounding` word was present in the `word_index` mapping still tokenizer represented it with `<OOV>`.

In [156]:
sample_text = "What happens when I add a number 2.1 in this sentence!"
sample_sequence = tokenizer.texts_to_sequences([sample_text])

print(f"Text: {sample_text}\n\n")
print(f"Tokenized text: {tokenizer.sequences_to_texts(sample_sequence)}")
print(f"Sequence: {sample_sequence}")

Text: What happens when I add a number 2.1 in this sentence!


Tokenized text: ['what happens when i add a number <OOV> in this sentence !']
Sequence: [[70, 2087, 54, 27, 1951, 7, 260, 1, 10, 38, 1074, 301]]


In [157]:
sample_text = "What happens when I add parenthesis (I am inside it!)."
sample_sequence = tokenizer.texts_to_sequences([sample_text])

print(f"Text: {sample_text}\n\n")
print(f"Tokenized text: {tokenizer.sequences_to_texts(sample_sequence)}")
print(f"Sequence: {sample_sequence}")

Text: What happens when I add parenthesis (I am inside it!).


Tokenized text: ['what happens when i add <OOV> ( i am inside it ! ) .']
Sequence: [[70, 2087, 54, 27, 1951, 1, 50, 27, 320, 514, 19, 301, 49, 2]]
